In [1]:
from google.cloud import bigquery
from functions import *
# Numpy & Pandas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns

import warnings

# Set the warning filter to ignore all warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=FutureWarning)

from sklearn import metrics
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, BaggingRegressor
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SequentialFeatureSelector

from scipy.stats import chi2_contingency

In [2]:
# set display options to show all columns
pd.set_option('display.max_columns', None)
# Set the float format to display numbers without scientific notation
pd.options.display.float_format = '{:.2f}'.format
# Set the client for future queries to BigQuery
client = bigquery.Client(project = "continente-lced-feup")
#data_table.enable_dataframe_formatter()
#auth.authenticate_user()

In [3]:
!gcloud auth application-default login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=K3LzqBwdFuJ9CwyZSfjImrsm80IQau&access_type=offline&code_challenge=ybl_KCGpfbWr2ZFvFmLt0x5Z6raLb3IGgv-scgusBo8&code_challenge_method=S256


Credentials saved to file: [/Users/vp/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "continente-lced-feup" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the r

In [4]:
query = client.query("""
   SELECT *
   FROM 
       tables_staging.df_models
  LIMIT 2000000
   """)

df = query.result().to_dataframe() # Wait for the job to complete.
df

,CUSTOMER_ACCOUNT_NR_MASK,SUBCAT_CD_EXT,MONTH,YEAR,QUARTER,SEMESTER,CUST_NUM_TRANSACTIONS_30_DAYS,CUST_NUM_TRANSACTIONS_90_DAYS,CUST_NUM_TRANSACTIONS_180_DAYS,CUST_NUM_TRANSACTIONS_360_DAYS,CUST_TOTAL_QTY_BOUGHT_30_DAYS,CUST_TOTAL_QTY_BOUGHT_90_DAYS,CUST_TOTAL_QTY_BOUGHT_180_DAYS,CUST_TOTAL_QTY_BOUGHT_360_DAYS,CUST_NUM_UNIQUE_SUBCAT_30_DAYS,CUST_NUM_UNIQUE_SUBCAT_90_DAYS,CUST_NUM_UNIQUE_SUBCAT_180_DAYS,CUST_NUM_UNIQUE_SUBCAT_360_DAYS,CUST_AVG_DAYS_FOR_NEXT_TRANSACTION_30_DAYS,CUST_AVG_DAYS_FOR_NEXT_TRANSACTION_90_DAYS,CUST_AVG_DAYS_FOR_NEXT_TRANSACTION_180_DAYS,CUST_AVG_DAYS_FOR_NEXT_TRANSACTION_360_DAYS,REG_AVG_DAYS_FOR_NEXT_TRANSACTION_30_DAYS,CUST_AVG_BASKET_SIZE_30_DAYS,CUST_AVG_BASKET_SIZE_90_DAYS,CUST_AVG_BASKET_SIZE_180_DAYS,CUST_AVG_BASKET_SIZE_360_DAYS,SUBCAT_NUM_TRANSACTIONS_30_DAYS,SUBCAT_NUM_TRANSACTIONS_90_DAYS,SUBCAT_NUM_TRANSACTIONS_180_DAYS,SUBCAT_NUM_TRANSACTIONS_360_DAYS,SUBCAT_TOTAL_QTY_BOUGHT_30_DAYS,SUBCAT_TOTAL_QTY_BOUGHT_90_DAYS,SUBCAT_TOTAL_QTY_BOUGHT_180_DAYS,SUBCAT_TOTAL_QTY_BOUGHT_360_DAYS,SUBCAT_NUM_UNIQUE_CUST_30_DAYS,SUBCAT_NUM_UNIQUE_CUST_90_DAYS,SUBCAT_NUM_UNIQUE_CUST_180_DAYS,SUBCAT_NUM_UNIQUE_CUST_360_DAYS,CUSTSUBCAT_NUM_TRANSACTIONS_30_DAYS,CUSTSUBCAT_NUM_TRANSACTIONS_90_DAYS,CUSTSUBCAT_NUM_TRANSACTIONS_180_DAYS,CUSTSUBCAT_NUM_TRANSACTIONS_360_DAYS,CUSTSUBCAT_TOTAL_QTY_BOUGHT_30_DAYS,CUSTSUBCAT_TOTAL_QTY_BOUGHT_90_DAYS,CUSTSUBCAT_TOTAL_QTY_BOUGHT_180_DAYS,CUSTSUBCAT_TOTAL_QTY_BOUGHT_360_DAYS,CUSTSUBCAT_AVG_DAYS_FOR_NEXT_TRANSACTION_30_DAYS,CUSTSUBCAT_AVG_DAYS_FOR_NEXT_TRANSACTION_90_DAYS,CUSTSUBCAT_AVG_DAYS_FOR_NEXT_TRANSACTION_180_DAYS,CUSTSUBCAT_AVG_DAYS_FOR_NEXT_TRANSACTION_360_DAYS,GENDER,FAMILY_MEMBERS,SEG_LIFESTYLE_CD,SEG_LIFESTAGE_CD,CAT_CD_EXT,TARGET
0,34953237,80403,1,2021,1,1,3,3,3,3,58,58,58,58,23,23,23,23,12,12,12,12,26.00,9.33,9.33,9.33,9.33,55,55,55,55,1793,1793,1793,1793,828,828,828,828,0,0,0,0,0,0,0,0,0,0,0,0,F,"(3, 8)",2,3,804,0
1,83613417,80403,1,2021,1,1,1,1,1,1,27,27,27,27,20,20,20,20,30,60,180,360,0.00,25.00,25.00,25.00,25.00,55,55,55,55,1793,1793,1793,1793,828,828,828,828,0,0,0,0,0,0,0,0,0,0,0,0,F,"(3, 8)",2,3,804,0
2,12474758,60102,1,2022,1,1,5,12,23,30,115,298,575,747,37,48,69,74,6,7,8,8,7.00,18.40,19.92,20.22,20.10,117,282,524,994,1245,3069,6024,11265,981,1876,2710,3487,0,0,0,0,1,1,2,2,0,0,0,0,F,"(3, 8)",2,4,601,0
3,25788875,140301,1,2022,1,1,9,22,33,44,119,284,551,717,32,54,77,84,3,3,5,8,5.00,8.78,8.45,11.00,10.91,47,144,267,515,1744,4955,9330,18689,979,1873,2534,3439,0,0,0,0,1,1,1,1,0,0,0,0,F,"(3, 8)",2,5,1403,0
4,3390331,140301,1,2022,1,1,4,8,13,31,76,136,195,520,23,30,36,51,8,12,13,11,11.00,9.00,8.38,7.54,7.39,47,144,267,515,1744,4955,9330,18689,979,1873,2534,3439,0,0,0,0,0,0,0,0,0,0,0,0,F,"(3, 8)",2,3,1403,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999995,101384874,10205,4,2022,2,1,2,7,9,22,7,62,97,221,5,27,34,53,11,12,21,16,30.00,3.50,6.00,7.00,6.36,571,1802,3565,6440,2348,7532,15171,27527,1240,2588,3680,4571,0,0,0,0,0,0,0,0,0,0,0,0,M,"(0, 0)",2,4,102,0
1999996,35323790,80111,3,2021,1,1,1,5,5,5,48,57,57,57,1,7,7,7,30,20,20,20,0.00,1.00,2.00,2.00,2.00,3,8,8,8,28,136,136,136,14,50,50,50,0,0,0,0,0,0,0,0,0,0,0,0,M,"(0, 0)",3,3,801,0
1999997,29331083,80111,3,2021,1,1,3,7,7,7,29,37,37,37,14,18,18,18,3,13,13,13,5.00,9.33,5.14,5.14,5.14,3,8,8,8,28,136,136,136,14,50,50,50,0,0,0,0,0,0,0,0,0,0,0,0,F,"(3, 8)",1,5,801,0
1999998,19204079,170310,4,2021,2,1,3,12,14,14,14,121,156,156,8,45,48,48,13,7,7,7,30.00,4.00,7.83,8.71,8.71,11,18,22,22,101,301,392,392,68,184,227,227,0,0,0,0,0,0,0,0,0,0,0,0,F,"(3, 8)",2,5,1703,0


# Data Preparation

In [5]:
df = df.dropna()
df = df.drop_duplicates()

df['fulldate'] = pd.to_datetime(df['MONTH'].astype(str) + '-' + df['YEAR'].astype(str))
# change from dtype datetime64[ns] to date time month
df['fulldate'] = df['fulldate'].dt.to_period('M')

df_200k = df.groupby('fulldate')#.apply(lambda x: x.drop(columns=['CUSTOMER_ACCOUNT_NR_MASK']))
df_200k = df.groupby('fulldate').sample(frac=0.5, random_state=101)

#drop customer id column because we want to generalize the model, instead of trying to predict for each customer
df_200k = df_200k.drop(columns=['CUSTOMER_ACCOUNT_NR_MASK'])

In [6]:
df_200k.value_counts('fulldate').sort_index()

fulldate
2021-01    36484
2021-02    39904
2021-03    43554
2021-04    44480
2021-05    43583
2021-06    46435
2021-07    49804
2021-08    45461
2021-09    24366
2021-10    27658
2021-11    26060
2021-12    30759
2022-01    50510
2022-02    50692
2022-03    51880
2022-04    52602
2022-05    55036
2022-06    55794
2022-07    54449
2022-08    48434
2022-09    27988
2022-10    31592
2022-11    29574
Freq: M, dtype: int64

In [7]:
numerical_columns = ['CUST_NUM_TRANSACTIONS_30_DAYS', 'CUST_NUM_TRANSACTIONS_90_DAYS',
       'CUST_NUM_TRANSACTIONS_180_DAYS', 'CUST_NUM_TRANSACTIONS_360_DAYS',
       'CUST_NUM_UNIQUE_SUBCAT_30_DAYS', 'CUST_NUM_UNIQUE_SUBCAT_90_DAYS',
       'CUST_NUM_UNIQUE_SUBCAT_180_DAYS', 'CUST_NUM_UNIQUE_SUBCAT_360_DAYS',
       'CUST_AVG_DAYS_FOR_NEXT_TRANSACTION_30_DAYS',
       'CUST_AVG_DAYS_FOR_NEXT_TRANSACTION_90_DAYS',
       'CUST_AVG_DAYS_FOR_NEXT_TRANSACTION_180_DAYS',
       'CUST_AVG_DAYS_FOR_NEXT_TRANSACTION_360_DAYS',
       'CUST_AVG_BASKET_SIZE_30_DAYS', 'CUST_AVG_BASKET_SIZE_90_DAYS',
       'CUST_AVG_BASKET_SIZE_180_DAYS', 'CUST_AVG_BASKET_SIZE_360_DAYS',
       'SUBCAT_NUM_TRANSACTIONS_30_DAYS', 'SUBCAT_NUM_TRANSACTIONS_90_DAYS',
       'SUBCAT_NUM_TRANSACTIONS_180_DAYS', 'SUBCAT_NUM_TRANSACTIONS_360_DAYS',
       'SUBCAT_NUM_UNIQUE_CUST_30_DAYS', 'SUBCAT_NUM_UNIQUE_CUST_90_DAYS',
       'SUBCAT_NUM_UNIQUE_CUST_180_DAYS', 'SUBCAT_NUM_UNIQUE_CUST_360_DAYS',
       'CUSTSUBCAT_NUM_TRANSACTIONS_30_DAYS',
       'CUSTSUBCAT_NUM_TRANSACTIONS_90_DAYS',
       'CUSTSUBCAT_NUM_TRANSACTIONS_180_DAYS',
       'CUSTSUBCAT_NUM_TRANSACTIONS_360_DAYS',
       'CUSTSUBCAT_AVG_DAYS_FOR_NEXT_TRANSACTION_30_DAYS',
       'CUSTSUBCAT_AVG_DAYS_FOR_NEXT_TRANSACTION_90_DAYS',
       'CUSTSUBCAT_AVG_DAYS_FOR_NEXT_TRANSACTION_180_DAYS',
       'CUSTSUBCAT_AVG_DAYS_FOR_NEXT_TRANSACTION_360_DAYS',
       'REG_AVG_DAYS_FOR_NEXT_TRANSACTION_30_DAYS']

#scaler = MinMaxScaler()
scaler = StandardScaler()
df_200k[numerical_columns] = scaler.fit_transform(df_200k[numerical_columns])

In [8]:
df_200k = df_200k.drop(columns=['QUARTER','SEMESTER','CAT_CD_EXT','SEG_LIFESTYLE_CD', 'YEAR', 'MONTH', 'SUBCAT_CD_EXT'])

In [9]:
# One-hot encoding categorical variables
df_200k = pd.get_dummies(df_200k, columns=['GENDER','SEG_LIFESTAGE_CD', 'FAMILY_MEMBERS'])
df_200k

,CUST_NUM_TRANSACTIONS_30_DAYS,CUST_NUM_TRANSACTIONS_90_DAYS,CUST_NUM_TRANSACTIONS_180_DAYS,CUST_NUM_TRANSACTIONS_360_DAYS,CUST_TOTAL_QTY_BOUGHT_30_DAYS,CUST_TOTAL_QTY_BOUGHT_90_DAYS,CUST_TOTAL_QTY_BOUGHT_180_DAYS,CUST_TOTAL_QTY_BOUGHT_360_DAYS,CUST_NUM_UNIQUE_SUBCAT_30_DAYS,CUST_NUM_UNIQUE_SUBCAT_90_DAYS,CUST_NUM_UNIQUE_SUBCAT_180_DAYS,CUST_NUM_UNIQUE_SUBCAT_360_DAYS,CUST_AVG_DAYS_FOR_NEXT_TRANSACTION_30_DAYS,CUST_AVG_DAYS_FOR_NEXT_TRANSACTION_90_DAYS,CUST_AVG_DAYS_FOR_NEXT_TRANSACTION_180_DAYS,CUST_AVG_DAYS_FOR_NEXT_TRANSACTION_360_DAYS,REG_AVG_DAYS_FOR_NEXT_TRANSACTION_30_DAYS,CUST_AVG_BASKET_SIZE_30_DAYS,CUST_AVG_BASKET_SIZE_90_DAYS,CUST_AVG_BASKET_SIZE_180_DAYS,CUST_AVG_BASKET_SIZE_360_DAYS,SUBCAT_NUM_TRANSACTIONS_30_DAYS,SUBCAT_NUM_TRANSACTIONS_90_DAYS,SUBCAT_NUM_TRANSACTIONS_180_DAYS,SUBCAT_NUM_TRANSACTIONS_360_DAYS,SUBCAT_TOTAL_QTY_BOUGHT_30_DAYS,SUBCAT_TOTAL_QTY_BOUGHT_90_DAYS,SUBCAT_TOTAL_QTY_BOUGHT_180_DAYS,SUBCAT_TOTAL_QTY_BOUGHT_360_DAYS,SUBCAT_NUM_UNIQUE_CUST_30_DAYS,SUBCAT_NUM_UNIQUE_CUST_90_DAYS,SUBCAT_NUM_UNIQUE_CUST_180_DAYS,SUBCAT_NUM_UNIQUE_CUST_360_DAYS,CUSTSUBCAT_NUM_TRANSACTIONS_30_DAYS,CUSTSUBCAT_NUM_TRANSACTIONS_90_DAYS,CUSTSUBCAT_NUM_TRANSACTIONS_180_DAYS,CUSTSUBCAT_NUM_TRANSACTIONS_360_DAYS,CUSTSUBCAT_TOTAL_QTY_BOUGHT_30_DAYS,CUSTSUBCAT_TOTAL_QTY_BOUGHT_90_DAYS,CUSTSUBCAT_TOTAL_QTY_BOUGHT_180_DAYS,CUSTSUBCAT_TOTAL_QTY_BOUGHT_360_DAYS,CUSTSUBCAT_AVG_DAYS_FOR_NEXT_TRANSACTION_30_DAYS,CUSTSUBCAT_AVG_DAYS_FOR_NEXT_TRANSACTION_90_DAYS,CUSTSUBCAT_AVG_DAYS_FOR_NEXT_TRANSACTION_180_DAYS,CUSTSUBCAT_AVG_DAYS_FOR_NEXT_TRANSACTION_360_DAYS,TARGET,fulldate,GENDER_F,GENDER_M,SEG_LIFESTAGE_CD_1,SEG_LIFESTAGE_CD_2,SEG_LIFESTAGE_CD_3,SEG_LIFESTAGE_CD_4,SEG_LIFESTAGE_CD_5,SEG_LIFESTAGE_CD_6,"FAMILY_MEMBERS_(0, 0)","FAMILY_MEMBERS_(1, 2)","FAMILY_MEMBERS_(3, 8)"
637025,1.17,-0.36,-0.77,-0.88,167,167,167,167,1.22,0.16,-0.30,-0.58,-0.78,-0.70,-0.44,-0.31,-0.80,0.67,0.78,0.81,0.81,-0.34,-0.38,-0.38,-0.36,427,427,427,427,-0.53,-0.93,-1.08,-1.16,-0.12,-0.11,-0.09,-0.09,0,0,0,0,-0.13,-0.13,-0.12,-0.11,0,2021-01,0,1,0,0,0,1,0,0,0,0,1
1127883,1.55,-0.21,-0.68,-0.84,44,44,44,44,-0.00,-0.69,-1.00,-1.18,-0.90,-0.78,-0.48,-0.33,-0.80,-0.55,-0.62,-0.66,-0.68,-0.12,-0.30,-0.34,-0.34,791,791,791,791,-0.17,-0.74,-0.93,-1.04,-0.12,-0.11,-0.09,-0.09,0,0,0,0,-0.13,-0.13,-0.12,-0.11,0,2021-01,1,0,0,0,0,0,1,0,0,1,0
882148,0.79,-0.51,-0.85,-0.93,59,59,59,59,0.69,-0.20,-0.60,-0.83,-0.67,-0.62,-0.41,-0.28,-0.43,0.18,0.22,0.22,0.22,-0.37,-0.39,-0.38,-0.37,143,143,143,143,-0.93,-1.14,-1.23,-1.29,-0.12,-0.11,-0.09,-0.09,0,0,0,0,-0.13,-0.13,-0.12,-0.11,0,2021-01,0,1,0,0,0,0,1,0,0,0,1
1053519,-1.10,-1.27,-1.26,-1.16,10,10,10,10,-0.88,-1.30,-1.51,-1.61,2.26,3.85,6.45,7.11,0.91,-0.04,-0.03,-0.04,-0.05,-0.41,-0.40,-0.39,-0.37,31,31,31,31,-1.13,-1.25,-1.31,-1.35,-0.12,-0.11,-0.09,-0.09,0,0,0,0,-0.13,-0.13,-0.12,-0.11,0,2021-01,0,1,0,0,1,0,0,0,0,0,1
1970745,0.41,-0.66,-0.93,-0.98,141,141,141,141,1.74,0.53,0.00,-0.32,-0.45,-0.46,-0.33,-0.24,-0.19,1.25,1.45,1.51,1.52,-0.40,-0.40,-0.39,-0.37,54,55,55,55,-1.10,-1.23,-1.30,-1.34,-0.12,-0.11,-0.09,-0.09,0,0,0,0,-0.13,-0.13,-0.12,-0.11,0,2021-01,1,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213989,-1.10,-0.66,-0.77,-0.88,1,17,31,31,-1.31,-0.93,-1.00,-1.18,2.26,0.74,0.30,0.09,-0.80,-1.02,-0.71,-0.66,-0.68,2.16,2.25,2.33,2.53,2545,7574,14675,28763,1.74,1.96,2.07,2.22,-0.12,-0.11,-0.09,-0.09,0,0,0,0,-0.13,-0.13,-0.12,-0.11,1,2022-11,1,0,0,0,0,1,0,0,0,0,1
575640,-0.35,-0.51,-0.35,0.18,42,69,86,136,0.35,-0.02,-0.10,-0.06,0.12,-0.14,0.06,-0.12,-1.04,0.18,-0.06,-0.48,-0.82,-0.41,-0.40,-0.39,-0.37,54,148,310,572,-1.09,-1.16,-1.16,-1.13,-0.12,-0.11,-0.09,-0.09,0,0,0,0,-0.13,-0.13,-0.12,-0.11,0,2022-11,1,0,0,0,1,0,0,0,0,0,1
646686,-0.73,-0.66,-0.27,0.41,4,14,52,104,-1.05,-1.05,-0.30,0.20,-0.33,0.81,-0.02,-0.16,

In [10]:
df_200k = df_200k.drop('TARGET', axis=1)

# Machine Learning

In [11]:
results_df = pd.DataFrame()

In [12]:
def random_oversampling(x_train,y_train):
    ros = RandomOverSampler(random_state=101)
    X_train, y_train = ros.fit_resample(x_train, y_train)
    
    return X_train, y_train

def roSMOTE(x_train,y_train):
    os = SMOTE(random_state=101)
    X_train, y_train = os.fit_resample(x_train, y_train)
    y_train = y_train.astype(np.float64)
    X_train = y_train.astype(np.float64)
    return X_train, y_train

def random_undersampling(x_train,y_train):
    rus = RandomUnderSampler(random_state=101)
    X_train, y_train = rus.fit_resample(x_train, y_train)
    
    return X_train, y_train

In [13]:
def reg_metrics(pred_model, x_train, x_test, y_train, y_test):
    """ Function takes in training and testing sets, prediction model, 
    and ouputs the below metrics:
    1. R² or Coefficient of Determination.
    2. Adjusted R²
    3. Mean Squared Error(MSE)
    4. Root-Mean-Squared-Error(RMSE).
    5. Mean-Absolute-Error(MAE).
    """
    global r2
    global adj_r2
    global mse
    global rmse
    global mae
    
    # Get predicted values on x_test
    y_pred = pred_model.predict(x_test)

    #1 & 2 Coefficient of Determination (R² & Adjusted R²)
    print("\n\t--- Coefficient of Determination (R² & Adjusted R²) ---")
    r2 = metrics.r2_score(y_pred=y_pred, y_true=y_test)
    adj_r2 = 1 - (1-r2)*(len(y_train)-1)/(len(y_train)-x_train.shape[1]-1)

    print(f"R²\t\t: {r2:.2f}")
    print(f"Adjusted R²\t: {adj_r2:.2f}")

    #3 & 4. MSE and RMSE
    print("\n\t--- Mean Squared Error (MSE & RMSE) ---")
   
    mse = metrics.mean_squared_error(y_pred=y_pred, y_true=y_test, squared=True)
    rmse = metrics.mean_squared_error(y_pred=y_pred, y_true=y_test, squared=False)
    
    print(f"MSE\t: {mse:.2f}")
    print(f"RMSE\t: {rmse:.2f}")

    #5. MAE
    print("\n\t--- Mean Absolute Error (MAE) ---")
    mae = metrics.mean_absolute_error(y_pred=y_pred, y_true=y_test)
    print(f"RMSE\t: {mae:.2f}")
    
    # # Return Accuracy
    # #train_acc = "{:.2f}".format(pred_model.score(x_train, y_train)*100)
    # #test_acc = "{:.2f}".format(pred_model.score(x_test, y_test)*100)
    
    # #Convert variables to a rounded format 
    # r2_formatted = "{:.2f}%".format(r2 * 100)
    # adj_r2_formatted = "{:.2f}%".format(adj_r2 * 100)
    # mse_formatted = "{:.2f}".format(mse)
    # rmse_formatted = "{:.2f}".format(rmse)
    # mae_formatted = "{:.2f}".format(mae)
    
    # # Return metrics as a dictionary
    # metrics_dict = {
    #     'R²': r2_formatted,
    #     'Adjusted R²': adj_r2_formatted,
    #     'MSE': mse_formatted,
    #     'RMSE': rmse_formatted,
    #     'MAE': mae_formatted
    # }
    
    #return metrics_dict

In [14]:
def cross_val_reg(df_ML, model, resampling_tecnique, model_name):
    global results_df
    print(f'\t------- {model_name} -------\n')

    timestamps = df_ML['fulldate'].sort_values().unique()

    # store each of the metrics in an array for each one
    rsquared_array = np.array([])
    adjrsquared_array = np.array([])
    mse_array = np.array([])
    rmse_array = np.array([])
    mae_array = np.array([])

    i = 0
    ii = 1
    max_timestamp = timestamps.max() - 2
    while timestamps[i] < max_timestamp:
            print('\n')
            print('-' * 50)
            print(f'\n\n\t---- Start Iteration {ii} ----\n')
            print("\n\t----- Iteration Info -----")
            print(f'\nTreino feito com os meses: {timestamps[i]} até {timestamps[i]+2}') # 11   
            train = df_ML[df_ML['fulldate'].isin([timestamps[i], timestamps[i]+1, timestamps[i]+2])]
            print(f'Teste feito com o mês: {timestamps[i]+3}\n')

            # test and train split
            train_tune = train.groupby('fulldate').sample(frac=0.3, random_state=101)
            X_train_cv = train.drop(['REG_AVG_DAYS_FOR_NEXT_TRANSACTION_30_DAYS','fulldate'], axis=1)
            y_train_cv = train['REG_AVG_DAYS_FOR_NEXT_TRANSACTION_30_DAYS'].astype(int)
            X_train_tune = train_tune.drop(['REG_AVG_DAYS_FOR_NEXT_TRANSACTION_30_DAYS','fulldate'], axis=1)
            y_train_tune = train_tune['REG_AVG_DAYS_FOR_NEXT_TRANSACTION_30_DAYS'].astype(int) 
                           
            # resampling technique
            X_train_cv, y_train_cv = resampling_tecnique(X_train_cv, y_train_cv)
            X_train_tune, y_train_tune = random_undersampling(X_train_tune, y_train_tune)

            print('X_train_cv shape: ', X_train_cv.shape)
            print('X_train_tune shape: ', X_train_tune.shape)
            
            test = df_ML[df_ML['fulldate'] == timestamps[i]+3] # 12
            
            X_test = test.drop(['REG_AVG_DAYS_FOR_NEXT_TRANSACTION_30_DAYS', 'fulldate'], axis=1)
            y_test = test['REG_AVG_DAYS_FOR_NEXT_TRANSACTION_30_DAYS'].astype(int)
            print('X_test shape: ', X_test.shape)
            print('\n\t--------------------------')
            # predictions for metrics
            y_pred = model(X_train_cv, y_train_cv, X_train_tune, y_train_tune, X_test, y_test)
            
            #append rsquared to rsquared_array
            rsquared_array = np.append(rsquared_array, r2)
            #append adjrsquared to ajdrsquared_array
            adjrsquared_array = np.append(adjrsquared_array, adj_r2)
            #append mse to mse_array
            mse_array = np.append(mse_array, mse)
            #append rmse to rmse_array
            rmse_array = np.append(rmse_array, mse)
            #append mae to mae_array
            mae_array = np.append(mae_array, mae)
            
            i += 1
            ii += 1
    else:
        print('\n----------------------------\n\n')
        print(f'R² Mean of all iterations : {np.mean(rsquared_array)}')
        print(f'Adjusted R² Mean of all iterations : {np.mean(adjrsquared_array)}')
        print(f'MSE Mean of all iterations: {np.mean(mse_array)}')
        print(f'RMSE mean of all iterations: {np.mean(rmse_array)}')
        print(f'MAE mean of all iterations: {np.mean(mae_array)}')

        # Append the results to the dataframe
        results_df = results_df.append({
        'Model': model_name,  # Replace 'Model Name' with the actual model name
        'R² Mean': np.mean(rsquared_array),
        'Adjusted R² Mean': np.mean(adjrsquared_array),
        'MSE Mean': np.mean(mse_array),
        'RMSE Mean': np.mean(rmse_array),
        'MAE Mean': np.mean(mae_array)}, ignore_index=True)
        
    return X_train_cv, y_train_cv,X_test, y_test, y_pred, results_df 



In [15]:
def LR(X_train, y_train, X_train_tune, y_train_tune, X_test, y_test):
    clf = Pipeline([
                # ('pca', PCA()),
                ('clf', LinearRegression())])
    
    param_grid = {}

    # Use SequentialFeatureSelector for forward/backward selection
    sfs_backward = SequentialFeatureSelector(clf, n_features_to_select=0.5, direction='forward', scoring='r2', cv=None, n_jobs=-1) # Does 5-Fold CV

    # Fit the feature selector to the training data
    sfs_backward.fit(X_train_tune, y_train_tune)

    # Get the selected features and transform the data
    X_train = sfs_backward.transform(X_train)
    X_test = sfs_backward.transform(X_test)

    print('\n\nNumber of selected features: {}'.format(sfs_backward.n_features_to_select_))
    #print('Features selected: {}'.format(sfs_backward.feature_names_in_[sfs_backward.support_]))
        
    gs = GridSearchCV(estimator=clf, param_grid=param_grid, scoring='r2', cv=2, verbose=1, n_jobs=-1)
    LR = Pipeline([('gs', gs)]) 
        
    LR.fit(X_train,y_train)
        
    reg_metrics(LR, X_train, X_test, y_train, y_test)

In [16]:
def ridge(X_train, y_train, X_train_tune, y_train_tune, X_test, y_test):
    clf = Pipeline([
                # ('pca', PCA()),
                ('clf', Ridge(max_iter=1000, tol=11.830e-06))])  

    param_grid = {
    'clf__alpha': (np.arange(0.001,1.0,0.005))}
    #'clf__alpha': [0.01]}

    # Use SequentialFeatureSelector for forward/backward selection
    sfs_backward = SequentialFeatureSelector(clf['clf'], n_features_to_select=0.5, direction='forward', scoring='r2', cv=None, n_jobs=-1) # Does 5-Fold CV

    # Fit the feature selector to the training data
    sfs_backward.fit(X_train_tune, y_train_tune)

    # Get the selected features and transform the data
    X_train = sfs_backward.transform(X_train)
    X_test = sfs_backward.transform(X_test)

    print('Number of selected features: {}'.format(sfs_backward.n_features_to_select_))
    #print('Features selected: {}'.format(sfs_backward.feature_names_in_[sfs_backward.support_]))
        
    gs = GridSearchCV(estimator=clf, param_grid=param_grid, scoring='r2', cv=2, verbose=1, n_jobs=-1)
    RG = Pipeline([('gs', gs)]) 
        
    RG.fit(X_train,y_train)
    
    # get best parameters
    best_params = gs.best_params_
    print('\n')
    print('-' * 60)
    print('Best Parameters:')
    print(best_params)
    print('-' * 60)
    print('\n')

    reg_metrics(RG, X_train, X_test, y_train, y_test)

In [17]:
def lasso(X_train, y_train, X_train_tune, y_train_tune, X_test, y_test):
    clf = Pipeline([
                # ('pca', PCA()),
                ('clf', Lasso(max_iter=1000, tol=11.830e-06))])  

    param_grid = {
    'clf__alpha': (np.arange(0.001,1.0,0.005))}

    # Use SequentialFeatureSelector for forward/backward selection
    sfs_backward = SequentialFeatureSelector(clf, n_features_to_select=0.5, direction='forward', scoring='r2', cv=None, n_jobs=-1) # Does 5-Fold CV

    # Fit the feature selector to the training data
    sfs_backward.fit(X_train_tune, y_train_tune)

    # Get the selected features and transform the data
    X_train = sfs_backward.transform(X_train)
    X_test = sfs_backward.transform(X_test)

    print('Number of selected features: {}'.format(sfs_backward.n_features_to_select_))
    #print('Features selected: {}'.format(sfs_backward.feature_names_in_[sfs_backward.support_]))
        
    gs = GridSearchCV(estimator=clf, param_grid=param_grid, scoring='r2', cv=2, verbose=1, n_jobs=-1)
    LS = Pipeline([('gs', gs)]) 
        
    LS.fit(X_train,y_train)

    # get best parameters
    best_params = gs.best_params_
    print('\n')
    print('-' * 60)
    print('Best Parameters:')
    print(best_params)
    print('-' * 60)
    print('\n')

    reg_metrics(LS, X_train, X_test, y_train, y_test)


In [18]:
def ElasticNetRegression(X_train, y_train, X_train_tune, y_train_tune, X_test, y_test):
    clf = Pipeline([
        # ('pca', PCA()),
        ('clf', ElasticNet(max_iter=10000,tol=11.830e-06))])

    param_grid = {
        'clf__alpha': (np.arange(0.001,1.0,0.005))
    }

    # Use SequentialFeatureSelector for forward/backward selection
    sfs_backward = SequentialFeatureSelector(
        clf['clf'], n_features_to_select=0.5, direction='forward', scoring='neg_mean_squared_error', cv=None,
        n_jobs=-1)  # Does 5-Fold CV

    # Fit the feature selector to the training data
    sfs_backward.fit(X_train_tune, y_train_tune)

    # Get the selected features and transform the data
    X_train = sfs_backward.transform(X_train)
    X_test = sfs_backward.transform(X_test)

    print('Number of selected features: {}'.format(sfs_backward.n_features_to_select_))

    gs = GridSearchCV(estimator=clf, param_grid=param_grid, scoring='neg_mean_squared_error', cv=2, verbose=1,
                      n_jobs=-1)
    EN = Pipeline([('gs', gs)])

    EN.fit(X_train, y_train)

    best_params = gs.best_params_
    print('\n')
    print('-' * 60)
    print('Best Parameters:')
    print(best_params)
    print('-' * 60)
    print('\n')
    
    reg_metrics(EN, X_train, X_test, y_train, y_test)

In [19]:
def gradientboostingreg(X_train, y_train, X_train_tune, y_train_tune, X_test, y_test):
    clf = Pipeline([
    # ('pca', PCA()),
    ('clf', GradientBoostingRegressor())])

    param_grid = {
        'clf__alpha': (np.arange(0.001,1.0,0.005))
    }

    # Use SequentialFeatureSelector for forward/backward selection
    sfs_backward = SequentialFeatureSelector(
        clf['clf'], n_features_to_select=0.5, direction='forward', scoring='r2', cv=None,
        n_jobs=-1)  # Does 5-Fold CV

    # Fit the feature selector to the training data
    sfs_backward.fit(X_train_tune, y_train_tune)

    # Get the selected features and transform the data
    X_train = sfs_backward.transform(X_train)
    X_test = sfs_backward.transform(X_test)

    print('Number of selected features: {}'.format(sfs_backward.n_features_to_select_))

    gs = GridSearchCV(estimator=clf, param_grid=param_grid, scoring='r2', cv=2, verbose=1,
                      n_jobs=-1)
    GB = Pipeline([('gs', gs)])

    GB.fit(X_train, y_train)

    best_params = gs.best_params_
    print('\n')
    print('-' * 60)
    print('Best Parameters:')
    print(best_params)
    print('-' * 60)
    print('\n')
    
    reg_metrics(GB, X_train, X_test, y_train, y_test)

In [20]:
def randomforestreg(X_train, y_train, X_train_tune, y_train_tune, X_test, y_test):
    clf = Pipeline([
    # ('pca', PCA()),
    ('clf', RandomForestRegressor())])

    param_grid = {
        'clf__min_samples_split': (range(1,10,1)),
        'clf__min_samples_leaf': (range(1,10,1))
    }

    # Use SequentialFeatureSelector for forward/backward selection
    sfs_backward = SequentialFeatureSelector(
        clf['clf'], n_features_to_select=0.5, direction='forward', scoring='r2', cv=None,
        n_jobs=-1)  # Does 5-Fold CV

    # Fit the feature selector to the training data
    sfs_backward.fit(X_train_tune, y_train_tune)

    # Get the selected features and transform the data
    X_train = sfs_backward.transform(X_train)
    X_test = sfs_backward.transform(X_test)

    print('Number of selected features: {}'.format(sfs_backward.n_features_to_select_))

    gs = GridSearchCV(estimator=clf, param_grid=param_grid, scoring='r2', cv=2, verbose=1,
                      n_jobs=-1)
    GB = Pipeline([('gs', gs)])

    GB.fit(X_train, y_train)

    best_params = gs.best_params_
    print('\n')
    print('-' * 60)
    print('Best Parameters:')
    print(best_params)
    print('-' * 60)
    print('\n')
    
    reg_metrics(GB, X_train, X_test, y_train, y_test)

## Random Oversampling

In [21]:
X_train_cv, y_train_cv,X_test, y_test, y_pred, results = cross_val_reg(df_200k, LR, random_oversampling, 'Linear Regression - OS')

	------- Linear Regression - OS -------



--------------------------------------------------


	---- Start Iteration 1 ----


	----- Iteration Info -----

Treino feito com os meses: 2021-01 até 2021-03
Teste feito com o mês: 2021-04

X_train_cv shape:  (362344, 55)
X_train_tune shape:  (5652, 55)
X_test shape:  (44480, 55)

	--------------------------


Number of selected features: 27
Fitting 2 folds for each of 1 candidates, totalling 2 fits

	--- Coefficient of Determination (R² & Adjusted R²) ---
R²		: -1539419398227248545792.00
Adjusted R²	: -1539534116660196933632.00

	--- Mean Squared Error (MSE & RMSE) ---
MSE	: 864642986778506428416.00
RMSE	: 29404812306.47

	--- Mean Absolute Error (MAE) ---
RMSE	: 2092749462.88


--------------------------------------------------


	---- Start Iteration 2 ----


	----- Iteration Info -----

Treino feito com os meses: 2021-02 até 2021-04
Teste feito com o mês: 2021-05

X_train_cv shape:  (381248, 55)
X_train_tune shape:  (6792, 55)
X_test sha

In [22]:
X_train_cv, y_train_cv,X_test, y_test, y_pred, results = cross_val_reg(df_200k, ridge, random_oversampling, 'Ridge Regression - OS')

	------- Ridge Regression - OS -------



--------------------------------------------------


	---- Start Iteration 1 ----


	----- Iteration Info -----

Treino feito com os meses: 2021-01 até 2021-03
Teste feito com o mês: 2021-04

X_train_cv shape:  (362344, 55)
X_train_tune shape:  (5652, 55)
X_test shape:  (44480, 55)

	--------------------------
Number of selected features: 27
Fitting 2 folds for each of 200 candidates, totalling 400 fits


------------------------------------------------------------
Best Parameters:
{'clf__alpha': 0.001}
------------------------------------------------------------



	--- Coefficient of Determination (R² & Adjusted R²) ---
R²		: -0.22
Adjusted R²	: -0.22

	--- Mean Squared Error (MSE & RMSE) ---
MSE	: 0.69
RMSE	: 0.83

	--- Mean Absolute Error (MAE) ---
RMSE	: 0.67


--------------------------------------------------


	---- Start Iteration 2 ----


	----- Iteration Info -----

Treino feito com os meses: 2021-02 até 2021-04
Teste feito com o mês

/Users/vp/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=6.20065e-17): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/Users/vp/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=6.3129e-17): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T




------------------------------------------------------------
Best Parameters:
{'clf__alpha': 0.001}
------------------------------------------------------------



	--- Coefficient of Determination (R² & Adjusted R²) ---
R²		: -0.35
Adjusted R²	: -0.35

	--- Mean Squared Error (MSE & RMSE) ---
MSE	: 0.51
RMSE	: 0.71

	--- Mean Absolute Error (MAE) ---
RMSE	: 0.58


--------------------------------------------------


	---- Start Iteration 9 ----


	----- Iteration Info -----

Treino feito com os meses: 2021-09 até 2021-11
Teste feito com o mês: 2021-12

X_train_cv shape:  (240428, 55)
X_train_tune shape:  (3800, 55)
X_test shape:  (30759, 55)

	--------------------------
Number of selected features: 27
Fitting 2 folds for each of 200 candidates, totalling 400 fits


/Users/vp/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=3.35223e-17): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/Users/vp/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=3.38014e-17): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T




------------------------------------------------------------
Best Parameters:
{'clf__alpha': 0.001}
------------------------------------------------------------



	--- Coefficient of Determination (R² & Adjusted R²) ---
R²		: -0.22
Adjusted R²	: -0.22

	--- Mean Squared Error (MSE & RMSE) ---
MSE	: 0.61
RMSE	: 0.78

	--- Mean Absolute Error (MAE) ---
RMSE	: 0.61


--------------------------------------------------


	---- Start Iteration 10 ----


	----- Iteration Info -----

Treino feito com os meses: 2021-10 até 2021-12
Teste feito com o mês: 2022-01

X_train_cv shape:  (260240, 55)
X_train_tune shape:  (3836, 55)
X_test shape:  (50510, 55)

	--------------------------
Number of selected features: 27
Fitting 2 folds for each of 200 candidates, totalling 400 fits


------------------------------------------------------------
Best Parameters:
{'clf__alpha': 0.001}
------------------------------------------------------------



	--- Coefficient of Determination (R² & Adjusted R²) ---

...



------------------------------------------------------------
Best Parameters:
{'clf__alpha': 0.001}
------------------------------------------------------------



	--- Coefficient of Determination (R² & Adjusted R²) ---
R²		: -0.28
Adjusted R²	: -0.28

	--- Mean Squared Error (MSE & RMSE) ---
MSE	: 0.65
RMSE	: 0.80

	--- Mean Absolute Error (MAE) ---
RMSE	: 0.66


--------------------------------------------------


	---- Start Iteration 19 ----


	----- Iteration Info -----

Treino feito com os meses: 2022-07 até 2022-09
Teste feito com o mês: 2022-10

X_train_cv shape:  (373596, 55)
X_train_tune shape:  (6304, 55)
X_test shape:  (31592, 55)

	--------------------------
Number of selected features: 27
Fitting 2 folds for each of 200 candidates, totalling 400 fits


/Users/vp/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=7.52775e-17): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/Users/vp/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=1.25463e-17): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/Users/vp/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=1.21929e-17): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/Users/vp/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=7.31572e-17): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T




------------------------------------------------------------
Best Parameters:
{'clf__alpha': 0.001}
------------------------------------------------------------



	--- Coefficient of Determination (R² & Adjusted R²) ---
R²		: -0.29
Adjusted R²	: -0.29

	--- Mean Squared Error (MSE & RMSE) ---
MSE	: 0.67
RMSE	: 0.82

	--- Mean Absolute Error (MAE) ---
RMSE	: 0.68


--------------------------------------------------


	---- Start Iteration 20 ----


	----- Iteration Info -----

Treino feito com os meses: 2022-08 até 2022-10
Teste feito com o mês: 2022-11

X_train_cv shape:  (313424, 55)
X_train_tune shape:  (5408, 55)
X_test shape:  (29574, 55)

	--------------------------
Number of selected features: 27
Fitting 2 folds for each of 200 candidates, totalling 400 fits


/Users/vp/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=6.12529e-17): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/Users/vp/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=9.88941e-18): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/Users/vp/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=1.02088e-17): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/Users/vp/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=1.08783e-16): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/Users/vp/anaconda3/lib/python3.9/site-packages/sklearn/linear_m



------------------------------------------------------------
Best Parameters:
{'clf__alpha': 0.001}
------------------------------------------------------------



	--- Coefficient of Determination (R² & Adjusted R²) ---
R²		: -0.36
Adjusted R²	: -0.36

	--- Mean Squared Error (MSE & RMSE) ---
MSE	: 0.58
RMSE	: 0.76

	--- Mean Absolute Error (MAE) ---
RMSE	: 0.63

----------------------------


R² Mean of all iterations : -0.2533837630036594
Adjusted R² Mean of all iterations : -0.2534764133547033
MSE Mean of all iterations: 0.6732424774903448
RMSE mean of all iterations: 0.6732424774903448
MAE mean of all iterations: 0.6736116308654027


In [23]:
X_train_cv, y_train_cv,X_test, y_test, y_pred, results = cross_val_reg(df_200k, lasso, random_oversampling, 'Lasso Regression - OS')

	------- Lasso Regression - OS -------



--------------------------------------------------


	---- Start Iteration 1 ----


	----- Iteration Info -----

Treino feito com os meses: 2021-01 até 2021-03
Teste feito com o mês: 2021-04

X_train_cv shape:  (362344, 55)
X_train_tune shape:  (5652, 55)
X_test shape:  (44480, 55)

	--------------------------
Number of selected features: 27
Fitting 2 folds for each of 200 candidates, totalling 400 fits


------------------------------------------------------------
Best Parameters:
{'clf__alpha': 0.996}
------------------------------------------------------------



	--- Coefficient of Determination (R² & Adjusted R²) ---
R²		: -0.27
Adjusted R²	: -0.27

	--- Mean Squared Error (MSE & RMSE) ---
MSE	: 0.71
RMSE	: 0.84

	--- Mean Absolute Error (MAE) ---
RMSE	: 0.73


--------------------------------------------------


	---- Start Iteration 2 ----


	----- Iteration Info -----

Treino feito com os meses: 2021-02 até 2021-04
Teste feito com o mês

In [24]:
X_train_cv, y_train_cv,X_test, y_test, y_pred, results = cross_val_reg(df_200k, ElasticNetRegression, random_oversampling, 'ElasticNet Regression - OS')

	------- ElasticNet Regression - OS -------



--------------------------------------------------


	---- Start Iteration 1 ----


	----- Iteration Info -----

Treino feito com os meses: 2021-01 até 2021-03
Teste feito com o mês: 2021-04

X_train_cv shape:  (362344, 55)
X_train_tune shape:  (5652, 55)
X_test shape:  (44480, 55)

	--------------------------
Number of selected features: 27
Fitting 2 folds for each of 200 candidates, totalling 400 fits


------------------------------------------------------------
Best Parameters:
{'clf__alpha': 0.486}
------------------------------------------------------------



	--- Coefficient of Determination (R² & Adjusted R²) ---
R²		: -0.27
Adjusted R²	: -0.27

	--- Mean Squared Error (MSE & RMSE) ---
MSE	: 0.71
RMSE	: 0.84

	--- Mean Absolute Error (MAE) ---
RMSE	: 0.73


--------------------------------------------------


	---- Start Iteration 2 ----


	----- Iteration Info -----

Treino feito com os meses: 2021-02 até 2021-04
Teste feito com 

In [25]:
X_train_cv, y_train_cv,X_test, y_test, y_pred, results = cross_val_reg(df_200k, gradientboostingreg, random_oversampling, 'Gradient Boosting Regression - OS')

	------- Gradient Boosting Regression - OS -------



--------------------------------------------------


	---- Start Iteration 1 ----


	----- Iteration Info -----

Treino feito com os meses: 2021-01 até 2021-03
Teste feito com o mês: 2021-04

X_train_cv shape:  (362344, 55)
X_train_tune shape:  (5652, 55)
X_test shape:  (44480, 55)

	--------------------------
Number of selected features: 27
Fitting 2 folds for each of 200 candidates, totalling 400 fits


------------------------------------------------------------
Best Parameters:
{'clf__alpha': 0.751}
------------------------------------------------------------



	--- Coefficient of Determination (R² & Adjusted R²) ---
R²		: -0.20
Adjusted R²	: -0.20

	--- Mean Squared Error (MSE & RMSE) ---
MSE	: 0.67
RMSE	: 0.82

	--- Mean Absolute Error (MAE) ---
RMSE	: 0.65


--------------------------------------------------


	---- Start Iteration 2 ----


	----- Iteration Info -----

Treino feito com os meses: 2021-02 até 2021-04
Teste fei

In [ ]:
X_train_cv, y_train_cv,X_test, y_test, y_pred, results = cross_val_reg(df_200k, randomforestreg, random_oversampling, 'Random Forest Regression - OS')

	------- Random Forest Regression - OS -------



--------------------------------------------------


	---- Start Iteration 1 ----


	----- Iteration Info -----

Treino feito com os meses: 2021-01 até 2021-03
Teste feito com o mês: 2021-04

X_train_cv shape:  (7332, 55)
X_train_tune shape:  (116, 55)
X_test shape:  (890, 55)

	--------------------------


## Random Undersampling

In [ ]:
X_train_cv, y_train_cv,X_test, y_test, y_pred, results = cross_val_reg(df_200k, LR, random_oversampling, 'Linear Regression - US')

In [ ]:
X_train_cv, y_train_cv,X_test, y_test, y_pred, results = cross_val_reg(df_200k, ridge, random_oversampling, 'Ridge Regression - US')

In [ ]:
X_train_cv, y_train_cv,X_test, y_test, y_pred, results = cross_val_reg(df_200k, lasso, random_oversampling, 'Ridge Regression - US')

In [ ]:
X_train_cv, y_train_cv,X_test, y_test, y_pred, results = cross_val_reg(df_200k, ElasticNetRegression, random_oversampling, 'Ridge Regression - US')

In [ ]:
X_train_cv, y_train_cv,X_test, y_test, y_pred, results = cross_val_reg(df_200k, gradientboostingreg, random_oversampling, 'Gradient Boosting Regression - US')

In [ ]:
X_train_cv, y_train_cv,X_test, y_test, y_pred, results = cross_val_reg(df_200k, randomforestreg, random_oversampling, 'Random Forest Regression - US')

## SMOTE